### 📘 Video 4: Make AI Search the Web (LangChain for Beginners, Part 4)

**Goal**: Let your AI search the web for up-to-date answers — no API keys, all free.

✅ Uses DuckDuckGo (no API key)

✅ Combines search + local LLM

✅ Runs on free Colab GPU

🌐 Let's make your AI know what's happening *right now*!

In [5]:
# Install required libraries
!pip install -q langchain-huggingface langchain-community duckduckgo-search transformers torch accelerate bitsandbytes
!pip install -U ddgs

### 🌐 Why Add Web Search?

Local models like TinyLlama have **no knowledge after their training date**.

To answer:
- "Who won the latest Eurovision?"
- "What's NVIDIA's stock price today?"

…you need **real-time data**.

👉 This is how production AI systems stay current.

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

# Load model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

# Create pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.7,
    do_sample=True
)

# Wrap with LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Model loaded!")

Device set to use cuda:0


✅ Model loaded!


### 🔍 Add Web Search Tool

In [7]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import Tool

# Initialize search
search = DuckDuckGoSearchAPIWrapper()

# Define search tool
search_tool = Tool(
    name="web_search",
    description="Search the web for current information",
    func=lambda query: search.results(query, max_results=3)
)

### 🧪 Ask Questions with Live Search

In [8]:
def answer_with_search(question):
    print(f"🔍 Searching: {question}")
    results = search_tool.run(question)

    # Format results
    context = "\n".join([f"[{i+1}] {r['title']}: {r['snippet']}" for i, r in enumerate(results)])

    prompt = f"<|im_start|>system\nYou are a helpful assistant. Answer based only on the search results below. Be concise.\n<|im_end|>\n<|im_start|>user\nSearch results:\n{context}\n\nQuestion: {question}<|im_end|>\n<|im_start|>assistant\n"
    response = llm.invoke(prompt)
    answer = response.replace(prompt, "").replace("<|im_start|>assistant\n", "").replace("<|im_end|>", "").strip()
    return answer

# Try it
q1 = "Who won the 2024 Eurovision Song Contest?"
print(f"Q: {q1}")
print(f"A: {answer_with_search(q1)}\n")

q2 = "What is the capital of Japan?"  # Simple fact
print(f"Q: {q2}")
print(f"A: {answer_with_search(q2)}")

Q: Who won the 2024 Eurovision Song Contest?
🔍 Searching: Who won the 2024 Eurovision Song Contest?
A: Answer: The 2024 Eurovision Song Contest was won by Switzerland with the song "The Code" performed by Nemo.

Q: What is the capital of Japan?
🔍 Searching: What is the capital of Japan?
A: According to the given search results, the capital of Japan is Tokyo. While no laws define the capital of Japan, it is widely accepted that Tokyo is the capital city of Japan.


In [9]:
q3 = "Where will the next soccer cup be held?"
print(f"Q: {q3}")
print(f"A: {answer_with_search(q3)}")

q4 = "What is an AI Agent?"
print(f"Q: {q4}")
print(f"A: {answer_with_search(q4)}")

Q: Where will the next soccer cup be held?
🔍 Searching: Where will the next soccer cup be held?
A: The next soccer cup will be held in 2026, which is the year when joint-hosts Mexico and the United States will host the 2026 FIFA World Cup.
Q: What is an AI Agent?
🔍 Searching: What is an AI Agent?
A: An AI Agent is an entity that can act autonomously in an environment by taking information from its surroundings, making decisions based on that data, and acting to transform circumstances. It is a type of intelligent agent in artificial intelligence that can perform tasks autonomously and learn from experience.


### 🎉 Summary

In this notebook, you:

✅ Gave your AI **real-time knowledge**

✅ Used `DuckDuckGoSearchAPIWrapper` (no API key!)

✅ Built a search-augmented AI

💡 This is **Retrieval-Augmented Generation (RAG) with live data** — used in enterprise AI systems

➡️ **Next: Make AI use tools — become a true agent!**

### 🔗 Resources

- [LangChain Tools Docs](https://python.langchain.com/docs/modules/agents/tools/)
- [DuckDuckGo Search Wrapper](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.duckduckgo_search.DuckDuckGoSearchAPIWrapper.html)
- [Author: Doug Ortiz](https://www.linkedin.com/in/doug-ortiz-illustris/)
- [YouTube Channel: @techbits-do](https://www.youtube.com/@techbits-do)
- [Illustris.org](https://www.illustris.org)